In [ ]:
import pandas as pd
from alpha_vantage.timeseries import TimeSeries
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go

In [ ]:
class ScriptData:
    API_KEY = "Z1RYTJSPV5DE01U7"

    def __init__(self):
        self.data = {}

    def fetch_intraday_data(self, symbol):
        # Fetch intraday data using Alpha Vantage API
        ts = TimeSeries(key=self.API_KEY, output_format='pandas')
        data, _ = ts.get_intraday(symbol=symbol, interval='1min')
        self.data[symbol] = data

    def fetch_historical_data(self, symbol, start_date, end_date):
        # Fetch historical data using Alpha Vantage API
        ts = TimeSeries(key=self.API_KEY, output_format='pandas')
        data, _ = ts.get_daily(symbol=symbol, outputsize='full')
        data = data.loc[start_date:end_date]
        self.data[symbol] = data

    def convert_intraday_data(self, symbol):
        # Convert intraday data to DataFrame format
        if symbol not in self.data:
            raise ValueError("Stock symbol not found. Call fetch_intraday_data() with the correct symbol.")
        
        df = pd.DataFrame({
            "timestamp": self.data[symbol].index,
            "open": self.data[symbol]['1. open'],
            "high": self.data[symbol]['2. high'],
            "low": self.data[symbol]['3. low'],
            "close": self.data[symbol]['4. close'],
            "volume": self.data[symbol]['5. volume']
        })

        return df

    def convert_historical_data(self, symbol):
        # Convert historical data to DataFrame format
        if symbol not in self.data:
            raise ValueError("Stock symbol not found. Call fetch_historical_data() with the correct symbol.")
        
        df = pd.DataFrame({
            "timestamp": self.data[symbol].index,
            "open": self.data[symbol]['1. open'],
            "high": self.data[symbol]['2. high'],
            "low": self.data[symbol]['3. low'],
            "close": self.data[symbol]['4. close'],
            "volume": self.data[symbol]['5. volume']
        })

        return df

    def __getitem__(self, key):
        if key not in self.data:
            raise ValueError("Stock symbol not found. Call fetch_intraday_data() or fetch_historical_data() with the correct symbol.")
        return self.convert_intraday_data(key)

    def __setitem__(self, key, value):
        raise NotImplementedError("Setting data by key is not supported.")

    def __contains__(self, key):
        return key in self.data



In [ ]:
# Create an instance of ScriptData
scrip_data = ScriptData()
# Fetch intraday data for GOOGL
scrip_data.fetch_intraday_data("GOOGL")
# Convert intraday data to DataFrame format for GOOGL
scrip_data.convert_intraday_data('GOOGL')
scrip_data['GOOGL']

In [ ]:
# Check if 'GOOGL' is in the ScriptData instance
'GOOGL' in scrip_data

In [ ]:
# Fetch intraday data for AAPL
scrip_data.fetch_intraday_data("AAPL")
# Convert intraday data to DataFrame format for AAPL
scrip_data.convert_intraday_data('AAPL')
scrip_data['AAPL']

In [ ]:
# Check if 'AAPL' is in the ScriptData instance
'AAPL' in scrip_data

In [ ]:
def indicator1(symbol, timeperiod):
    scrip_data = ScriptData()
    scrip_data.fetch_intraday_data(symbol)
    df = scrip_data.convert_intraday_data(symbol)
    df['indicator'] = df['close'].rolling(window=timeperiod, min_periods=1).mean()
    return df[['timestamp', 'indicator']]


In [ ]:
# Calculate indicator1 for GOOGL with a time period of 5
indicator1('GOOGL', timeperiod=5)

In [ ]:
# Calculate indicator1 for AAPL with a time period of 5
indicator1('AAPL', timeperiod=5)

In [ ]:
class Strategy:
    def __init__(self, script):
        self.script = script
        self.scrip_data = ScriptData()
        self.indicator_data = None
        self.signals = None
    
    def fetch_intraday_data(self):
        self.scrip_data.fetch_intraday_data(self.script)
    
    def compute_indicator_data(self):
        df = self.scrip_data.convert_intraday_data(self.script)
        df['indicator'] = df['close'].rolling(window=5, min_periods=1).mean()
        self.indicator_data = df[['timestamp', 'indicator']]
    
    def generate_signals(self):
        df = self.scrip_data.convert_intraday_data(self.script)
        df['indicator'] = self.indicator_data['indicator']
        df['signal'] = 'NO_SIGNAL'
        df.loc[df['close'] < df['indicator'], 'signal'] = 'BUY'
        df.loc[df['close'] > df['indicator'], 'signal'] = 'SELL'
        self.signals = df[['timestamp', 'close', 'signal']]  # Include the 'close' column in the signals DataFrame


    
    def run_strategy(self):
        self.fetch_intraday_data()
        self.compute_indicator_data()
        self.generate_signals()

In [ ]:
# Create an instance of Strategy for GOOGL
strategy = Strategy('GOOGL')
# Run the strategy
strategy.run_strategy()
# Get the signals DataFrame
signals = strategy.signals
signals

In [ ]:
# Filter the signals for 'BUY' and 'SELL' signals0
buy_sell_signals = signals[(signals['signal'] == 'BUY') | (signals['signal'] == 'SELL')]
buy_sell_signals

In [ ]:
# Get the GOOGL data
df = scrip_data['GOOGL']

# Convert the 'timestamp' column to a datetime index
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.set_index('timestamp', inplace=True)

# Resample the data to 10-minute intervals
df_resampled = df.resample('4T').agg({'open': 'first', 'high': 'max', 'low': 'min', 'close': 'last', 'volume': 'sum'})

# Calculate the 20-period SMA
sma_period = 20
df_resampled['sma'] = df_resampled['close'].rolling(sma_period).mean()

# Extend the SMA line to the start and end points of the chart
sma_extended = pd.Series([df_resampled['sma'].iloc[0]] * len(df_resampled.index), index=df_resampled.index)
df_resampled['sma_extended'] = sma_extended

# Create the candlestick chart
fig = go.Figure(data=[
    go.Candlestick(x=df_resampled.index,
                   open=df_resampled['open'],
                   high=df_resampled['high'],
                   low=df_resampled['low'],
                   close=df_resampled['close'],
                   name='Candlestick'),
    go.Scatter(x=df_resampled.index,
               y=df_resampled['sma_extended'],
               name=f'SMA {sma_period}',
               line=dict(color='black'))
])

# Update the chart layout
fig.update_layout(xaxis_rangeslider_visible=False,
                  xaxis=dict(type='category'),
                  yaxis_title='Price',
                  title='GOOGL Candlestick Chart')

# Display the chart
fig.show()

In [ ]:
# Fetch intraday data for NVDA
scrip_data.fetch_intraday_data("NVDA")

# Create an instance of Strategy for NVDA
strategy_nvda = Strategy('NVDA')
# Run the strategy
strategy_nvda.run_strategy()
# Get the signals DataFrame for NVDA
signals_nvda = strategy_nvda.signals

# Filter the signals for NVDA
nvda_buy_sell_signals = signals_nvda[(signals_nvda['signal'] == 'BUY') | (signals_nvda['signal'] == 'SELL')]

# Get the NVDA data
df_nvda = scrip_data['NVDA']

# Convert the 'timestamp' column to a datetime index for NVDA
df_nvda['timestamp'] = pd.to_datetime(df_nvda['timestamp'])
df_nvda.set_index('timestamp', inplace=True)

# Resample the data to 5-minute intervals for NVDA
df_resampled_nvda = df_nvda.resample('5T').agg({'open': 'first', 'high': 'max', 'low': 'min', 'close': 'last', 'volume': 'sum'})

# Create the candlestick chart for NVDA
fig_nvda = go.Figure(data=[
    go.Candlestick(x=df_resampled_nvda.index,
                   open=df_resampled_nvda['open'],
                   high=df_resampled_nvda['high'],
                   low=df_resampled_nvda['low'],
                   close=df_resampled_nvda['close'],
                   name='Candlestick'),
    go.Scatter(x=nvda_buy_sell_signals.index,
               y=nvda_buy_sell_signals['close'],
               mode='markers',
               marker=dict(symbol=nvda_buy_sell_signals['signal'].map({'BUY': 'triangle-up', 'SELL': 'triangle-down'}),
                           size=10,
                           color=nvda_buy_sell_signals['signal'].map({'BUY': 'green', 'SELL': 'red'}),
                           line=dict(color='black', width=1)),
               name='Signals')
])

# Update the chart layout for NVDA
fig_nvda.update_layout(xaxis_rangeslider_visible=False,
                       xaxis=dict(type='category'),
                       yaxis_title='Price',
                       title='NVDA Candlestick Chart with Signals')

# Display the chart for NVDA
fig_nvda.show()
